In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
from torch.nn.functional import one_hot

batch_size = 32
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=1)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=True, num_workers=1)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [4]:
import torch.nn as nn
import torch.nn.functional as F



class summary_layer(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(summary_layer, self).__init__()

        self.beta_I = torch.eye(in_dim,in_dim)
        self.beta_func1 = nn.Linear(in_dim,out_dim)
        
    def forward(self, x):

        beta = self.beta_func1(self.beta_I)
        
        x1 = x@beta
        x2 = x@beta@(beta.T@beta)

        return x1,x2

    
class ConceptNet(nn.Module):
    def __init__(self,num_classes):
        super(ConceptNet,self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.summary_layer1 = summary_layer(64*5*5, 120)
        self.output = nn.Linear(120,10)

    def forward(self, x):
        x = self.pool((self.conv1(x)))
        x = self.pool((self.conv2(x)))
        x1,x2 = self.summary_layer1(torch.flatten(x,1))
        return self.output(x1),self.output(x2)
        return x

    

def ConceptNet_simple(num_classes):
    return ConceptNet(num_classes)
    

In [5]:
import torch.optim as optim

net = ConceptNet_simple(10)

mseloss = nn.MSELoss()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [6]:
import time

start = time.time()

results = []

for epoch in range(15):  # loop over the dataset multiple times
    count=0
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        count=i
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs1,outputs2 = net(inputs)
        
        loss = criterion(outputs1, labels)+mseloss(outputs1, outputs2)+criterion(outputs2,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        # print statistics

    print(f'[{epoch + 1}, {count + 1:5d}] loss: {running_loss  :.10f}')
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            # calculate outputs by running images through the network
            outputs1,outputs2 = net(images)
            # the class with the highest energy is what we choose as prediction
            _, predicted = torch.max(outputs1.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')   
    
    
    end = time.time()
    print(end - start)
    results += [[epoch,end-start,running_loss,100*correct/total]]
    running_loss = 0.0
print('Finished Training')

end = time.time()
print(end - start)

[1,  1563] loss: 9372.5697004795
Accuracy of the network on the 10000 test images: 37.93 %
64.86551880836487
[2,  1563] loss: 5175.1413209438
Accuracy of the network on the 10000 test images: 46.33 %
128.3325958251953
[3,  1563] loss: 4570.1128584146
Accuracy of the network on the 10000 test images: 51.26 %
193.39263892173767
[4,  1563] loss: 4326.2869918346
Accuracy of the network on the 10000 test images: 53.29 %
259.25027894973755
[5,  1563] loss: 4146.4132741690
Accuracy of the network on the 10000 test images: 56.59 %
324.183429479599
[6,  1563] loss: 3975.3656990528
Accuracy of the network on the 10000 test images: 57.76 %
387.1988174915314
[7,  1563] loss: 3764.7616357803
Accuracy of the network on the 10000 test images: 60.28 %
451.6983757019043
[8,  1563] loss: 3617.3904693127
Accuracy of the network on the 10000 test images: 61.24 %
516.5967283248901
[9,  1563] loss: 3488.5791970491
Accuracy of the network on the 10000 test images: 60.66 %
581.2577514648438
[10,  1563] loss: 